In [48]:
from scipy.io import netcdf
import subprocess
from davitpy.pydarn.radar import *
from davitpy.pydarn.plotting import *
from davitpy.utils import *
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plot', 'pylab', 'figsize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [31]:
fName = "../data/sdr/f18/20141216/PS.APL_V0116S024CB0005_SC.U_DI.A_GP.F18-SSUSI_PA.APL-SDR-DISK_DD.20141216_SN.26619-02_DF.NC"

In [33]:
fh = netcdf.netcdf_file(fName, mmap=False)

lats = fh.variables['PIERCEPOINT_DAY_LATITUDE'].data
lons = fh.variables['PIERCEPOINT_DAY_LONGITUDE'].data
alts = fh.variables['PIERCEPOINT_DAY_ALTITUDE'].data

sc_lat = fh.variables['DMSP_LATITUDE'].data.ravel()
sc_lon = fh.variables['DMSP_LONGITUDE'].data.ravel()
sc_alt = fh.variables['DMSP_ALTITUDE'].data.ravel()


In [45]:
for va in fh.variables.keys():
    print va

TIME_DAY
TIME_EPOCH_DAY
YEAR_DAY
DOY_DAY
ORBIT_DAY
LATITUDE_DAY
LONGITUDE_DAY
ALTITUDE_DAY
TIME_DAY_AURORAL
TIME_EPOCH_DAY_AURORAL
YEAR_DAY_AURORAL
DOY_DAY_AURORAL
ORBIT_DAY_AURORAL
LATITUDE_DAY_AURORAL
LONGITUDE_DAY_AURORAL
ALTITUDE_DAY_AURORAL
TIME_NIGHT
TIME_EPOCH_NIGHT
YEAR_NIGHT
DOY_NIGHT
ORBIT_NIGHT
LATITUDE_NIGHT
LONGITUDE_NIGHT
ALTITUDE_NIGHT
PIERCEPOINT_NIGHT_LATITUDE
PIERCEPOINT_NIGHT_LONGITUDE
PIERCEPOINT_NIGHT_ALTITUDE
PIERCEPOINT_NIGHT_SZA
PIERCEPOINT_DAY_LATITUDE
PIERCEPOINT_DAY_LONGITUDE
PIERCEPOINT_DAY_ALTITUDE
PIERCEPOINT_DAY_SZA
PIERCEPOINT_DAY_LATITUDE_AURORAL
PIERCEPOINT_DAY_LONGITUDE_AURORAL
PIERCEPOINT_DAY_ALTITUDE_AURORAL
PIERCEPOINT_DAY_SZA_AURORAL
IN_SAA_NIGHT
IN_SAA_DAY
IN_SAA_DAY_AURORAL
ACROSSPIXELSIZE_DAY
ALONGPIXELSIZE_DAY
ACROSSPIXELSIZE_DAY_AURORAL
ALONGPIXELSIZE_DAY_AURORAL
ACROSSPIXELSIZE_NIGHT
ALONGPIXELSIZE_NIGHT
EFFECTIVELOOKANGLE_NIGHT
EFFECTIVELOOKANGLE_DAY
EFFECTIVELOOKANGLE_DAY_AURORAL
DARK_COUNT_CORRECTION
SCATTER_LIGHT_1216_CORRECTION
SCATTER_

In [42]:
data1 = fh.variables['DISK_INTENSITY_DAY'].data[:, :, 2]
data2 = fh.variables['DISK_INTENSITY_DAY'].data[:, :, 3]
data3 = fh.variables['DISK_INTENSITY_DAY'].data[:, :, 4]

In [47]:
print data1.shape
print lats.shape

(42, 623)
(42, 623)


In [ ]:
fig = figure(figsize=(10,10))
m = plotUtils.mapObj(boundinglat=30., coords='geo')

fig.savefig("../figs/ssusi-test.pdf",bbox_inches='tight')